# Netlist driven flow (circuits)

You can define a circuit by its YAML netlist thanks to `gdsfactory.read.from_yaml`

YAML is a more human readable version of JSON

to define a Component from YAML you need to define:

- instances: with each instance setting
- placements: with X and Y

Optionally you can also define:

- routes: between instances
- connections: to connect components ports
- ports: define input and output circuit ports


You have two options for working with gdsfactory:

1. **layout driven flow**: you code your layout using python functions, and then extract the YAML netlist to simulate the circuit. These is the flow that you have been doing so far.
2. **netlist driven flow**: you start with the YAML definition of your circuit (instances, placements and routes). From the netlist you can simulate the circuit or generate the layout.

Using the netlist driven flow you can define components, circuits and masks.

Lets see what you can do


You can run some YAML code and show it in Klayout.

In [ ]:
%%script gf yaml build 

name: mask_compact
pdk: ubcpdk

instances:
  rings:
    component: pack_doe
    settings:
      doe: ring_single
      settings:
        radius: [30, 50, 20, 40]
        length_x: [1, 2, 3]
      do_permutations: True
      function:
        function: add_fiber_array
        settings:
            fanout_length: 200


  mzis:
    component: pack_doe_grid
    settings:
      doe: mzi
      settings:
        delta_length: [10, 100]
      do_permutations: True
      spacing: [10, 10]
      function: add_fiber_array

placements:
  rings:
    xmin: 50

  mzis:
    xmin: rings,east


You can also define it in ipython widgets and update the mask both in klayout and matplotlib

In [ ]:
%matplotlib widget

import ipywidgets
from IPython.display import clear_output
import gdsfactory as gf

x = ipywidgets.Textarea(rows=20,columns=280)

x.value = """
name: sample_different_factory

instances:
    bl:
      component: pad
    tl:
      component: pad
    br:
      component: pad
    tr:
      component: pad

placements:
    tl:
        x: 200
        y: 500

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600


routes:
    electrical:
        settings:
            separation: 20
            layer: [31, 0]
            width: 10
        links:
            tl,e3: tr,e1
            bl,e3: br,e1
    optical:
        settings:
            radius: 100
        links:
            bl,e4: br,e3

"""

In [ ]:
display(x)
out = ipywidgets.Output()
gf.config.set_plot_options(show_subports=False, zoom_factor=2)

#@out.capture()
def f(change):
    c = gf.read.from_yaml(change['new'])
    #clear_output()
    c.plot()
    c.show()

x.observe(f, 'value')
f({'new':x.value})

## instances

Lets start by defining the `instances`

In [ ]:
import gdsfactory as gf

gf.CONF.plotter = "matplotlib"  # This notebook rendered with 'holoviews' exceeds the 100MB limit for github pages

yaml = """

instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
"""

c = gf.read.from_yaml(yaml)
c.plot()

You can modify the instances later (not recommended).

I recommend that you define that using `placements` in the `YAML` file

In [ ]:
c.instances["mmi_long"].x = 100
c.show()
c.plot()

## placements

Lets define the placecemts of each instance by defining a `placements` section in YAML

Lets place an `mmi_long` where you can place the `W0` port at `x=20, y=10`

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: o1
        x: 20
        y: 10
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

You can also mirror it

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: o1
        x: 20
        y: 10
        mirror: True
"""

c = gf.read.from_yaml(yaml)
c.plot()

## ports

You can export the ports of any instance to the new `component_from_yaml` Component.

You will need to define a `ports` section in YAML

Lets expose all the ports from `mmi_long` into the new component.

Ports are exposed as `new_port_name: instance_name, port_name`

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: o1
        x: 20
        y: 10
        mirror: True
        
ports:
    o3: mmi_long,o3
    o2: mmi_long,o2
    o1: mmi_long,o1
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

You can also define a mirror placement using a port

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: o1
"""

c = gf.read.from_yaml(yaml)
c.plot()

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 0
        y: 0
        mirror: 25
ports:
    o1: mmi_long,o3
    o2: mmi_long,o2
    o3: mmi_long,o1
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: o1
        x: 10
        y: 20
        rotation: 90
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        port: o1
        x: 10
        y: 20
        rotation: 90
ports:
    o1: mmi_long,o2
    o2: mmi_long,o3
    o3: mmi_long,o1
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

In [ ]:
c.size_info.north

In [ ]:
c.size_info.east

## connections

You can connect any two instances by defining a `connections` section in the YAML file.

it follows the syntax.

`instance_source,port : instance_destination,port`

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    b:
      component: bend_circular
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_short:
        port: o1
        x: 10
        y: 20
connections:
    b,o1 : mmi_short,o2
    mmi_long,o1: b, o2

ports:
    o1: mmi_short,o1
    o2: mmi_long,o2
    o3: mmi_long,o3
"""


c = gf.read.from_yaml(yaml)
c.show()
c.plot()

In [ ]:
from omegaconf import OmegaConf
import io

In [ ]:
d = OmegaConf.load(
    io.StringIO(
        """
x: mmi_short,o3 10
dx: 10
"""
    )
)
d

In [ ]:
d.keys()

**Relative port placing**

You can also place a component with respect to another instance port

You can also define an x and y offset with `dx` and `dy`

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: o1
        x: 0
        y: 0
    mmi_long:
        port: o1
        x: mmi_short,o2
        y: mmi_short,o2
        dx : 10
        dy: -10
"""


c = gf.read.from_yaml(yaml)
c.show()
c.plot()

**Cicular reference warning**

You have to be careful not doing a circuilar reference

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5

placements:
    mmi_short:
        port: o1
        x: mmi_long,o2
        y: mmi_long,o2
    mmi_long:
        port: o1
        x: mmi_short,o2
        y: mmi_short,o2
        dx : 10
        dy: 20
"""

c = gf.read.from_yaml(yaml)
c.plot()

## routes

You can define routes between two instanes by defining a `routes` section in YAML

it follows the syntax

```YAML

routes:
    route_name:
        links:
            instance_source,port: instance_destination,port
        settings:  # for the route (optional)
            waveguide: strip
            width: 1.2

```

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    optical:
        links:
            mmi_short,o2: mmi_long,o1
        settings:
            cross_section:
                cross_section: strip
                settings:
                    layer: [2, 0]
"""


c = gf.read.from_yaml(yaml)
c.show()
c.plot()

You can **rotate** and instance specifying the angle in degrees

You can also define ports for the component

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        links:
            mmi_short,o2: mmi_long,o3
    
ports:
    o1: mmi_short,o1
    o2: mmi_long,o1
    
"""

c = gf.read.from_yaml(yaml)
c.show()
c.plot()

You can also access the routes in the newly created component

In [ ]:
r = c.routes["mmi_short,o2:mmi_long,o3"]
r

As well as the instances

In [ ]:
c.instances

## instances, placements, connections, ports, routes

Lets combine all you learned so far.

You can define the netlist connections of a component by a netlist in YAML format

Note that you define the connections as `instance_source.port ->
instance_destination.port` so the order is important and therefore you can only
change the position of the `instance_destination`

For example, this coupler has the center coupling region at (100, 0)

In [ ]:
gf.components.coupler_symmetric()

In [ ]:
import gdsfactory as gf

gap = 0.2
wg_width = 0.5
length = 10

yaml = f"""
instances:
    left:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    right:
      component: coupler_symmetric
      settings:
        gap: {gap}
        width: {wg_width}
    center:
      component: coupler_straight
      settings:
        gap: {gap}
        width: {wg_width}
        length: {length}

placements:
    center:
        x: 100
        y: 0
    left:
        mirror: True

connections:
    center,o1: left,o1
    right,o1: center,o4

ports:
    o1: left,o4
    o2: left,o3
    o3: right,o3
    o4: right,o4

"""

c = gf.read.from_yaml(yaml)
c.plot()

While this one has the sbend_left_coupler `sl` centered at (100, 0)

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    route1:
        links:
            mmi_short,o2: mmi_long,o1
"""


c = gf.read.from_yaml(yaml)
c.plot()

You can **rotate** and instance specifying the angle in degrees

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
routes:
    mmi_connect:
        links:
            mmi_short,o2: mmi_long,o3
"""


c = gf.read.from_yaml(yaml)
c.plot()

You can also define ports for the component

In [ ]:
import gdsfactory as gf

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        links:
            mmi_short,o2: mmi_long,o3
    
ports:
    o1: mmi_short,o1
    o2: mmi_long,o1
"""

c = gf.read.from_yaml(yaml)
c.plot()

In [ ]:
c.routes

In [ ]:
r = c.routes["mmi_short,o2:mmi_long,o3"]

In [ ]:
c.instances

In [ ]:
c.routes

You can define several routes that will be connected using `gf.routing.get_bundle`

In [ ]:
import gdsfactory as gf

sample_2x2_connections_solution = """
name:
    connections_2x2_problem

instances:
    mmi_bottom:
      component: mmi2x2
    mmi_top:
      component: mmi2x2

placements:
    mmi_top:
        x: 100
        y: 100

routes:
    optical:
        links:
            mmi_bottom,o4: mmi_top,o1
            mmi_bottom,o3: mmi_top,o2

"""

c = gf.read.from_yaml(sample_2x2_connections_solution)
c.plot()

You can also add custom component_factories to `gf.read.from_yaml`

In [ ]:
@gf.cell
def pad_new(size=(100, 100), layer=gf.LAYER.WG):
    c = gf.Component()
    compass = c << gf.components.compass(size=size, layer=layer)
    c.ports = compass.ports
    return c


c = pad_new(cache=False)
c.plot()

In [ ]:
import gdsfactory as gf

factory = gf.components.cells

gf.get_active_pdk().register_cells(pad_new=pad_new)

sample_custom_component = """
name:
    connections_2x2_problem

instances:
    bot:
      component: pad_new
    top:
      component: pad_new

placements:
    top:
        x: 0
        y: 200
"""

c = gf.read.from_yaml(sample_custom_component)
c.plot()

In [ ]:
sample_custom_component = """
name:
    custom_routes

instances:
    t:
      component: pad_array
      settings:
          orientation: 270
          columns: 3
    b:
      component: pad_array
      settings:
          orientation: 90
          columns: 3

placements:
    t:
        x: 200
        y: 400
routes:
    electrical:
        settings:
            layer: [31, 0]
            width: 10.
            end_straight_length: 150
        links:
            t,e11: b,e11
            t,e13: b,e13
"""


c = gf.read.from_yaml(sample_custom_component)
c.plot()

Also, you can define route aliases, that have different settings and specify the route `factory` as a parameter as well as the `settings` for that particular route alias.

In [ ]:
import gdsfactory as gf

sample_settings = """
name: sample_settings

instances:
    bl:
      component: pad
    tl:
      component: pad
    br:
      component: pad
    tr:
      component: pad

placements:
    tl:
        x: 0
        y: 200

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600

routes:
    optical_r100:
        settings:
            radius: 100
            layer: [31, 0]
            width: 50
        links:
            tl,e2: tr,e2
    optical_r200:
        settings:
            radius: 200
            width: 10
            layer: [31, 0]
        links:
            bl,e3: br,e3
"""
c = gf.read.from_yaml(sample_settings)
c.plot()

In [ ]:
sample_custom_component = """

instances:
    t:
      component: pad_array
      settings:
          orientation: 270
          columns: 3
    b:
      component: pad_array
      settings:
          orientation: 90
          columns: 3

placements:
    t:
        x: 200
        y: 500
routes:
    optical:
        settings: 
            radius: 50
            width: 40
            layer: [31,0]
            end_straight_length: 150
            separation: 50
        links:
            t,e11: b,e11
            t,e12: b,e12
            t,e13: b,e13
"""

c = gf.read.from_yaml(sample_custom_component)
c.plot()

In [ ]:
import gdsfactory as gf

sample = """

instances:
    t:
      component: pad_array
      settings:
          orientation: 270
          columns: 3
    b:
      component: pad_array
      settings:
          orientation: 90
          columns: 3

placements:
    t:
        x: 100
        y: 1000
routes:
    route1:
        routing_strategy: get_bundle_path_length_match
        settings: 
            extra_length: 500
            width: 2
            layer: [31,0]
            end_straight_length: 500
        links:
            t,e11: b,e11
            t,e12: b,e12
"""

c = gf.read.from_yaml(sample)
print(c.routes["t,e11:b,e11"])
c.plot()

In [ ]:
import gdsfactory as gf

sample = """
instances:
    t:
      component: pad_array
      settings:
          orientation: 270
          columns: 3
    b:
      component: pad_array
      settings:
          orientation: 90
          columns: 3

placements:
    t:
        x: -250
        y: 1000
routes:
    route1:
        routing_strategy: get_bundle_from_waypoints
        settings:
            waypoints:
                - [0, 300]
                - [400, 300]
                - [400, 400]
                - [-250, 400]
            auto_widen: False
        links:
            b,e11: t,e11
            b,e12: t,e12

"""

c = gf.read.from_yaml(sample)
c.plot()

In [ ]:
from omegaconf import OmegaConf
import io

d = OmegaConf.load(
    io.StringIO(
        """
way_points: 
    - [0,0]
    - [0, 600]
    - [-250, 600]
    - [-250, 1000]

demo: a
"""
    )
)

In [ ]:
d = OmegaConf.load(
    io.StringIO("way_points:  [[0,0], [0, 600], [-250, 600], [-250, 1000]]")
)

In [ ]:
d

In [ ]:
import numpy as np

c = gf.Component("waypoints_sample")
route = gf.routing.get_route_from_waypoints(waypoints=np.array(d["way_points"]))

c.add(route.references)
c.plot()

Note that you define the connections as `instance_source.port -> instance_destination.port` so the order is important and therefore you can only change the position of the `instance_destination`

## Custom factories

You can leverage netlist defined components to define more complex circuits

In [ ]:
import gdsfactory as gf

mmi1x2_faba = gf.partial(gf.components.mmi1x2, length_mmi=30)
mmi2x2_faba = gf.partial(gf.components.mmi2x2, length_mmi=30)

yaml = """
name: sample_custom_cells
instances:
    mmit:
      component: mmi2x2_faba
    mmib:
      component: mmi1x2_faba
      settings:
        width_mmi: 4.5
placements:
    mmit:
        x: 100
        y: 100
routes:
    route1:
        links:
            mmib,o2: mmit,o2
            
ports:
    o1: mmib,o1
    o2: mmit,o2
    o3: mmit,o3
    o4: mmit,o4
"""

# Make sure all the components that you want to use are registered in the active PDK
gf.get_active_pdk().register_cells(mmi1x2_faba=mmi1x2_faba, mmi2x2_faba=mmi2x2_faba)
c = gf.read.from_yaml(yaml)
c

In [ ]:
import gdsfactory as gf


@gf.cell
def mzi_custom(length_x=0):
    netlist = f"""
instances:
    mzi:
        component: mzi_phase_shifter_top_heater_metal
        settings:
            length_x: 50
        
    pads:
        component: pad_array
        settings:
            columns: 2

placements:
    mzi:
        x: 0
    pads:
        y: 200
        x: 0
ports:
    o1: mzi,o1
    o2: mzi,o2
    
    
routes:
    electrical1:
        links:
            mzi,e1: pads,e11
        settings:
            layer: [41, 0]
            width: 10
            radius: 10
            
    electrical2:
        links:
            mzi,e2: pads,e12
            
        settings:
            layer: [41, 0]
            width: 10
            radius: 10

"""
    return gf.read.from_yaml(netlist)


c = mzi_custom(length_x=10, cache=False)
c.show()
c.plot()

In [ ]:
c = gf.components.mzi()

In [ ]:
c.plot_netlist()

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_dict().keys())

## variables

In [ ]:
import gdsfactory as gf

yaml = """

vars:
    length_mmi: 10

instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: ${vars.length_mmi}
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
"""

c = gf.read.from_yaml(yaml)
c.plot()

## get_netlist (Component -> YAML)

Any component exports its netlist `get_netlist` and returns an `OmegaConf` dict that can be easily converted into JSON and YAML.

While `component_from_yaml` converts YAML -> Component

`get_netlist` converts Component -> YAML

In [ ]:
import io
from omegaconf import OmegaConf
import gdsfactory as gf

In [ ]:
c = gf.components.ring_single()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
netlist = c.get_netlist()

In [ ]:
n = netlist

In [ ]:
c.write_netlist("ring.yml")

In [ ]:
n = OmegaConf.load("ring.yml")

In [ ]:
i = list(n["instances"].keys())
i

In [ ]:
instance_name0 = i[0]

In [ ]:
n["instances"][instance_name0]["settings"]

```python
import gdsfactory as gf
from omegaconf import OmegaConf
import pathlib

c1 = gf.read.from_yaml('ring.yml')
c1
```

```python
n = c1.get_netlist(full_settings=True)
connections = n['connections']
len(connections)
```

## Plot netlist

You can plot the netlist of components.

Every gdsfactory component can either be defined by its netlist or using layout friendly functions such as component sequence to define it and then `get_netlist()` method.

Connections are determined by extracting all the ports of a component, and asuming that ports with the same (x, y) are connected.

 When you do `get_netlist()` for a component it will only show connections for the instances that belong to that component (it trims the netlist). So despite having  a lot of connections, it will show only the meaningful connections for that component. For example, a ring has a ring_coupler. but if you want to digg deeper, the connections that made that ring coupler are still available.

In [ ]:
import gdsfactory as gf

In [ ]:
c = gf.components.mzi()
c.plot()

In [ ]:
c = gf.components.mzi()
n = c.get_netlist()
print(c.get_netlist_dict().keys())

In [ ]:
c.plot_netlist()

In [ ]:
n.keys()

In [ ]:
import gdsfactory as gf

yaml = """
name: mmu_with_bend
instances:

    mmi1x2_12_0:
        component: mmi1x2

    bend_circular_R10p00_32_4:
      component: bend_circular
    
    straight_L1p00_35_11:
        component: straight
        settings:
            length: 10
            layer: [2, 0]

connections:
    bend_circular_R10p00_32_4,o1: mmi1x2_12_0,o2
    straight_L1p00_35_11,o1: bend_circular_R10p00_32_4,o2
"""

c = gf.read.from_yaml(yaml)
c.show()
c.name = "mmi_with_bend_circular"
print(c.name)
c.plot()

In [ ]:
n = c.get_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
n["connections"]

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.mzi()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.ring_single()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.ring_double()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
import gdsfactory as gf

c = gf.components.ring_single()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.ring_double()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c = gf.components.mzi()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.mzit()
c.plot()

In [ ]:
c.plot_netlist()

In [ ]:
c = gf.components.mzi_lattice()
c.plot()

In [ ]:
import gdsfactory as gf

coupler_lengths = [10, 20, 30]
coupler_gaps = [0.1, 0.2, 0.3]
delta_lengths = [10, 100]

c = gf.components.mzi_lattice(
    coupler_lengths=coupler_lengths,
    coupler_gaps=coupler_gaps,
    delta_lengths=delta_lengths,
)
c.plot()

In [ ]:
print(c.get_netlist_yaml())

In [ ]:
c.plot_netlist()

In [ ]:
coupler_lengths = [10, 20, 30, 40]
coupler_gaps = [0.1, 0.2, 0.4, 0.5]
delta_lengths = [10, 100, 200]

c = gf.components.mzi_lattice(
    coupler_lengths=coupler_lengths,
    coupler_gaps=coupler_gaps,
    delta_lengths=delta_lengths,
)
c.plot()

In [ ]:
n = c.get_netlist()

In [ ]:
c.plot_netlist()